In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1. Cargar los datos 

In [2]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Directorios de datos
directorio_14 = PARENT_DIR / "14. Mallado Total"

df_mall_total = pd.read_csv(f"{directorio_14}/MALLADO_Total.csv", sep=',', encoding='latin-1')

In [3]:
# Definimos una función genérica para calcular la media ponderada
def weighted_mean(x, value_col, weight_col):
    w = x[weight_col]
    v = x[value_col]
    return np.nan if w.sum() == 0 else (v * w).sum() / w.sum()

# Agrupamos por fecha
df_agg = (
    df_mall_total.groupby(['Year', 'Month', 'Day'], as_index=False)
      .apply(lambda g: pd.Series({
          # Temperaturas ponderadas por población
          'Temp_min_C_Nat': weighted_mean(g, 'Temp_min_C', 'POBLACION_TOTAL'),
          'Temp_max_C_Nat': weighted_mean(g, 'Temp_max_C', 'POBLACION_TOTAL'),
          'Temp_media_C_Nat': weighted_mean(g, 'Temp_media_C', 'POBLACION_TOTAL'),
          # Radiación ponderada por potencia solar
          'RadiacionGlobal_media_Wm2_Nat': weighted_mean(g, 'RadiacionGlobal_media_Wm2', 'Pot_Solar_kW'),
          # Viento ponderado por potencia eólica
          'Viento_media_ms_Nat': weighted_mean(g, 'Viento_media_ms', 'Pot_Eolico_kW'),
      }))
      .reset_index(drop=True)
)

# Resultado final
df_agg['DayOfYear'] = pd.to_datetime({'year': df_agg['Year'], 'month': df_agg['Month'], 'day': df_agg['Day']}).dt.dayofyear


C:\Users\Elena\AppData\Local\Temp\ipykernel_13608\1146055454.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [4]:
# Guardar resultados
df_agg.to_csv(f"{BASE_DIR}/CLIMA_NACIONAL_POND.csv", index=False)